In [ ]:
import sys, os
import urllib.request
import pandas as pd
import zipfile
import logging
import time

%run ./common/file_utility
%run ./common/treaty_state

skip_columns = [ 'extra_entry', 'dbflag', 'french', 'other', 'regis', 'regisant', 'force', 'group1', 'group2' ]

state = TreatyState(skip_columns=skip_columns).process()

# TODO Lägg nedladdade filer i en ZIP-fil


In [ ]:


def filter_treaties(wti_treaties):
    #global processed_gof_treaties
    data = wti_treaties
    # processed_treaty_id = [ x[0] for x in processed_gof_treaties ]

    data = data.loc[
        (data.is_cultural==True)&
        (data.signed_year.between(1945,1972))&
        (data.source.isin(['UNTS', 'UNXX']))&
        (data.english!='en')&
        #(~data.index.isin(processed_treaty_id))&
        (data.ispartyof4!='Yes')
    ]
    return data


data = filter_treaties(state.treaties)

FileUtility.save_excel([(data, 'Data')], 'unt-log-page.xls')


In [ ]:

def fetch_pdf(url, pdf_path):
    try:
        response = urllib.request.urlopen(url)
        http_code = response.getcode()
        if http_code in [ 200 ]:
            pdf = response.read()
            with open(pdf_path,'wb') as output:
                output.write(pdf)
        return http_code
    except Exception as ex:
        #print(str(ex))
        #print(url)
        return 0

def get_candidate_names(treaty_id, volume, language):
    template = 'volume-{0}-{3}-{1}-{2}.pdf'
    return [
        template.format(volume, int(str(treaty_id)[-5:]), language, 'I'),
        template.format(volume, int(str(treaty_id)[-5:]), language, 'II')
    ]
        
def ts_data_path(data_dir, filename):
    return os.path.join(data_dir, '{}_{}'.format(time.strftime("%Y%m%d%H%M"), filename))

def scrape_treaty(treaty_id, volume, language, pdf_filename):
    url_template = 'https://treaties.un.org/doc/Publication/UNTS/Volume%20{0}/{1}'
    for candidate_name in get_candidate_names(treaty_id, volume, language):
        url = url_template.format(volume, candidate_name)
        http_code = fetch_pdf(url, pdf_filename)
        if http_code in [ 200 ]:
            return http_code, url
    return http_code, None

def scrape_treaties(data, languages):
    
    scrape_log = pd.DataFrame(columns=['treaty_id', 'volume', 'page', 'party1', 'party2', 'signed', 'language', 'http_code', 'url'])

    for treaty_id, treaty in data.iterrows():

        volume = treaty['vol']

        for (language, lang) in languages:

            pdf_filename = os.path.join('./data', "{}-{}.pdf".format(treaty_id, lang))

            if (os.path.isfile(pdf_filename)):
                print('Already scraped: {}'.format(pdf_filename))
                continue
                
            http_code, url = scrape_treaty(treaty_id, volume, language, pdf_filename)   

            print('{}/{};{};{};{}'.format(volume, treaty_id, language,  http_code, url or ''))

            scrape_log = scrape_log.append({
                    'treaty_id' : treaty_id,
                    'volume' : treaty['vol'],
                    'page' : treaty['page'],
                    'party1': treaty['party1'],
                    'party2': treaty['party2'],
                    'signed': treaty['signed'],
                    'language': language,
                    'http_code': http_code,
                    'url': url or ''
            }, ignore_index=True)

    return scrape_log


In [ ]:

languages = [ ('English', 'en') ] #, ('French', 'fr'), ('Other', 'other') ]

data = filter_treaties(state.treaties)

scrape_log = scrape_treaties(data, languages)

filename = ts_data_path('../data', 'unts-en-download-log.xlsx')
FileUtility.save_excel([(scrape_log, 'Data')], filename)


In [ ]:
%%bash
#sudo apt-get install poppler-utils
cd ./data
find -name '*.pdf' -exec pdftotext -eol dos "{}" \;